In [1]:
%%capture
!pip install SimpleITK

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import SimpleITK as sitk
import os
import time
from glob import glob
from tqdm import tqdm


def writeSlices(series_tag_values, new_img, i, out_dir):
    image_slice = new_img[:,:,i]

    # Convert floating type image (image_slice) to int type (imgFiltered)
    castFilter = sitk.CastImageFilter()
    castFilter.SetOutputPixelType(sitk.sitkInt16)
    imgFiltered = castFilter.Execute(image_slice)

    writer = sitk.ImageFileWriter()
    writer.KeepOriginalImageUIDOn()

    # Tags shared by the series.
    list(map(lambda tag_value: imgFiltered.SetMetaData(tag_value[0], tag_value[1]), series_tag_values))

    # Slice specific tags.
    imgFiltered.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
    imgFiltered.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time

    # Setting the type to CT preserves the slice location.
    imgFiltered.SetMetaData("0008|0060", "CT")  # set the type to CT so the thickness is carried over

    # (0020, 0032) image position patient determines the 3D spacing between slices.
    imgFiltered.SetMetaData("0020|0032", '\\'.join(map(str,new_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
    imgFiltered.SetMetaData("0020,0013", str(i)) # Instance Number

    # Write to the output directory and add the extension dcm, to force writing in DICOM format.
    writer.SetFileName(os.path.join(out_dir,'slice' + str(i).zfill(4) + '.dcm'))
    writer.Execute(imgFiltered)

def nifti2dicom_1file(in_dir, out_dir):
    """
    This function is to convert only one nifti file into dicom series

    `nifti_dir`: the path to the one nifti file
    `out_dir`: the path to output
    """

    os.makedirs(out_dir, exist_ok=True)

    new_img = sitk.ReadImage(in_dir)
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")

    direction = new_img.GetDirection()
    series_tag_values = [("0008|0031",modification_time), # Series Time
                    ("0008|0021",modification_date), # Series Date
                    ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                    ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                    ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                        direction[1],direction[4],direction[7])))),
                    ("0008|103e", "Created-Pycad")] # Series Description

    # Write slices to output directory
    list(map(lambda i: writeSlices(series_tag_values, new_img, i, out_dir), range(new_img.GetDepth())))

def nifti2dicom_mfiles(nifti_dir, out_dir=''):
    """
    This function is to convert multiple nifti files into dicom files

    `nifti_dir`: You enter the global path to all of the nifti files here.
    `out_dir`: Put the path to where you want to save all the dicoms here.

    PS: Each nifti file's folders will be created automatically, so you do not need to create an empty folder for each patient.
    """

    images = glob(nifti_dir + '/*.nii.gz')

    for image in images:
        o_path = out_dir + '/' + os.path.basename(image)[:-7]
        os.makedirs(o_path, exist_ok=True)

        nifti2dicom_1file(image, o_path)

In [6]:
for i in tqdm(range(1, 424)):
  format = str(i).zfill(3)
  # in_path = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/imagesTr/pancreas_'+ format + '.nii.gz'
  # out_path = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/images/pancreas_' + format
  in_path = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_'+ format + '.nii.gz'
  out_path = '/content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels/pancreas_' + format
  if os.path.exists(in_path):
    print(f"Converting {in_path} to DICOM...")
    nifti2dicom_1file(in_path, out_path)
    print(f"Conversion completed for {in_path}. Find your new DICOM images in {out_path}")
  else:
    print(f"Skipping {in_path} as the file does not exist.")

  0%|          | 0/423 [00:00<?, ?it/s]

Converting /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_001.nii.gz to DICOM...


  0%|          | 1/423 [00:02<15:04,  2.14s/it]

Conversion completed for /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_001.nii.gz. Find your new DICOM images in /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels/pancreas_001
Skipping /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_002.nii.gz as the file does not exist.
Skipping /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_003.nii.gz as the file does not exist.
Converting /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_004.nii.gz to DICOM...


  1%|          | 4/423 [00:03<06:11,  1.13it/s]

...

Conversion completed for /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_418.nii.gz. Find your new DICOM images in /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels/pancreas_418
Converting /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_419.nii.gz to DICOM...


 99%|█████████▉| 419/423 [08:11<00:05,  1.29s/it]

Conversion completed for /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_419.nii.gz. Find your new DICOM images in /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels/pancreas_419
Skipping /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_420.nii.gz as the file does not exist.
Converting /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_421.nii.gz to DICOM...


100%|██████████| 423/423 [08:14<00:00,  1.17s/it]

Conversion completed for /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_421.nii.gz. Find your new DICOM images in /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels/pancreas_421
Skipping /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_422.nii.gz as the file does not exist.
Skipping /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/labelsTr/pancreas_423.nii.gz as the file does not exist.


In [8]:
!ls /content/drive/MyDrive/Machine-Learning-Biomedicine/Pancreatic-Cancer/pancreas/dicom_files/labels

pancreas_001  pancreas_098  pancreas_194  pancreas_276	pancreas_350
pancreas_004  pancreas_099  pancreas_196  pancreas_277	pancreas_351
pancreas_005  pancreas_100  pancreas_197  pancreas_278	pancreas_354
pancreas_006  pancreas_101  pancreas_198  pancreas_279	pancreas_355
pancreas_010  pancreas_102  pancreas_199  pancreas_280	pancreas_356
pancreas_012  pancreas_103  pancreas_200  pancreas_283	pancreas_357
pancreas_015  pancreas_104  pancreas_201  pancreas_284	pancreas_358
pancreas_016  pancreas_105  pancreas_203  pancreas_285	pancreas_360
pancreas_018  pancreas_106  pancreas_204  pancreas_286	pancreas_361
pancreas_019  pancreas_107  pancreas_207  pancreas_287	pancreas_362
pancreas_021  pancreas_109  pancreas_209  pancreas_289	pancreas_364
pancreas_024  pancreas_110  pancreas_210  pancreas_290	pancreas_365
pancreas_025  pancreas_111  pancreas_211  pancreas_291	pancreas_366
pancreas_028  pancreas_113  pancreas_212  pancreas_292	pancreas_367
pancreas_029  pancreas_114  pancreas_213  pancre